# SMILES → Tanimoto score computed on all fingerprints used in MolForge
Calcula el Tanimoto coefficient entre dos columnes de **SMILES** fent servir les 15 **fingerprints** utilitzades al paper de MolForge

**Entrada**: CSV amb columnes `SMILES_input` i `SMILES_output_ECFP4`.

**Sortida**: CSV amb columnes `MACCS`, `Avalon`, `RDK4`, `RDK4-L`, `HashAP`, `TT`, `HashTT`, `ECFP0`, `ECFP2`, `ECFP4`, `FCFP2`, `FCFP4`, `AEs`, `ECFP2*`, `ECFP4*`.

## Imports

In [1]:
# Per definir l'arrel del projecte
import os

# Pandas pels dataframes
import pandas as pd
import numpy as np # pels NaN

# Per no mostrar els Warnings de RDKit
from rdkit import RDLogger
from rdkit import DataStructs
RDLogger.DisableLog("rdApp.*")

## Inputs (part a editar)

Arrel del projecte

In [2]:
os.chdir("/export/home/ddiestre/MolForge_Testing")

In [3]:
# Importem la funció smiles_to_fingerprint del nostre sourcecode que utilitza RDKit
from src.smiles_to_fp import smiles_to_fingerprint, get_supported_fingerprints
from src.fingerprints import FpSimilarity

Fingerprints en que transformar els SMILES

In [4]:
fp_type = "ECFP4"

Fitxer de fingerprints preprocessat (path a partir de MolForge_Testing/)

In [5]:
input_path = "data/MolForge_output/MolForge_MFoutput_2000_ECFP4.csv"
#input_path = "data/MolForge_output/CoCoGraph_MFoutput_2000_novel.csv"
#input_path = "data/MolForge_output/CoCoGraph_MFoutput_2000_lt70atoms.csv"

SMILES_in_col_name = "SMILES_input"
SMILES_out_col_name = "SMILES_output_" + fp_type

Fitxer en que guardar l'output (path a partir de MolForge_Testing/)

In [6]:
output_path = "data/analysis_output/MolForge_MF_Analysis_2000_ECFP4/"
#output_path = "data/analysis_output/CoCoGraph_MF_Analysis_2000_novel/"
#output_path = "data/analysis_output/CoCoGraph_MF_Analysis_2000_lt70atoms/"

# Columna on guardarem l'output de MolForge en format fp_type (per visualitzar els resultats en format bitvectors)
fp_out_col_name = "fingerprints_output_" + fp_type

## 1. Lectura del fitxer

In [8]:
# Lectura del fitxer
df = pd.read_csv(input_path, sep = ',', index_col = 0)
df.head(5)

,SMILES_input,fingerprints_input_ECFP4,SMILES_output_ECFP4
id,,,
1,CCCCCN(CCCCc1c2cc3sc2nn13)C(=O)c1ccc(OCC)c(Cl)c1,70 80 85 94 162 237 294 347 366 378 412 425 51...,CCCCCC1=C2C=C3C=C(C(=NN3C(=C2C=C4N1N=C5N4N=C6C...
2,O=C(Nc1n[nH]c2ccc(Br)cc12)N1CCCC1,74 91 119 122 218 328 369 378 650 708 728 766 ...,C1CCN(C1)C(=O)NC2=NNC3=C2C=C(C=C3)Br
3,CCN(C(=O)N(C)c1ccccc1OC)C(=Cc1ccc(OC)cc1)c1ccc...,25 54 80 249 252 294 322 376 431 650 694 695 7...,CCN(C1=CC=CC=C1OC)C(=O)N(C)C2=CC=CC=C2/C(=C/C3...
4,CC1(Cl)OC(=O)C1(C)Cl,99 113 314 650 656 667 1041 1057 1060 1135 127...,CC1(C2(C(C(=O)O1)(C(C(=O)O2)(Cl)Cl)Cl)C)C
5,CCc1cc(C=CC(C)=Cc2cc(OC)ccc2C)ccc1C,25 31 80 135 294 322 517 650 694 695 718 781 8...,CCC1=C(C=CC(=C1)/C=C/C(=C/C2=C(C=CC(=C2)OC)C)/C)C


## 2. Columna SMILES_output → Columna de fingerprints

Aquesta part del codi ens serveix únicament per visualitzar els resultats de MolForge en forma de fingerprints i per identificar les al·lucinacions.

In [9]:
fingerprints = []

# Apliquem el conversor SMILES -> fingerprint
fingerprints = df[SMILES_out_col_name].apply(
    lambda s: smiles_to_fingerprint(s, fp_type=fp_type, n_bits=2048, return_bits=True)
)

# Guardem les fingerprints en el mateix format que l'input de MolForge
df[fp_out_col_name] = fingerprints.apply(
    lambda lst: " ".join(str(x) for x in lst) if isinstance(lst, list) else lst
)

# Visusalitzem el nou dataframe
df.head(12)

,SMILES_input,fingerprints_input_ECFP4,SMILES_output_ECFP4,fingerprints_output_ECFP4
id,,,,
1,CCCCCN(CCCCc1c2cc3sc2nn13)C(=O)c1ccc(OCC)c(Cl)c1,70 80 85 94 162 237 294 347 366 378 412 425 51...,CCCCCC1=C2C=C3C=C(C(=NN3C(=C2C=C4N1N=C5N4N=C6C...,38 80 84 85 95 158 191 237 294 378 410 438 518...
2,O=C(Nc1n[nH]c2ccc(Br)cc12)N1CCCC1,74 91 119 122 218 328 369 378 650 708 728 766 ...,C1CCN(C1)C(=O)NC2=NNC3=C2C=C(C=C3)Br,74 91 119 122 218 328 369 378 650 708 728 766 ...
3,CCN(C(=O)N(C)c1ccccc1OC)C(=Cc1ccc(OC)cc1)c1ccc...,25 54 80 249 252 294 322 376 431 650 694 695 7...,CCN(C1=CC=CC=C1OC)C(=O)N(C)C2=CC=CC=C2/C(=C/C3...,25 54 80 181 249 294 322 376 557 650 694 695 7...
4,CC1(Cl)OC(=O)C1(C)Cl,99 113 314 650 656 667 1041 1057 1060 1135 127...,CC1(C2(C(C(=O)O1)(C(C(=O)O2)(Cl)Cl)Cl)C)C,86 113 149 256 314 438 488 650 656 726 780 915...
5,CCc1cc(C=CC(C)=Cc2cc(OC)ccc2C)ccc1C,25 31 80 135 294 322 517 650 694 695 718 781 8...,CCC1=C(C=CC(=C1)/C=C/C(=C/C2=C(C=CC(=C2)OC)C)/C)C,25 31 80 135 294 322 517 650 694 695 718 781 8...
6,CCc1ccc2ccc3[nH]c(=O)n(C)c(=O)[nH]c(=O)c4ccc(c...,2 80 119 145 203 294 310 314 345 437 552 564 6...,CCC1=CC2=C(C=C1)NC3=C2C=CC4=C3C5=C(C=C4)C6=C(C...,80 119 145 203 223 294 310 314 504 540 564 569...
7,CC(C)(C)C1C2CC3(CCCCC3)C3C(N2)C(C)(C)C2CCCCC2N13,2 16 101 114 293 322 380 392 431 519 524 549 5...,CC1([C@@H]2CCCC[C@H]2N3[C@H]1[C@H]4C5(CCCCC5)C...,2 16 85 237 261 293 299 380 519 549 579 663 67...
8,CC1(C)C(=O)NC(=O)N1c1ccc(C(F)(F)F)cc1Cl,114 228 314 366 379 561 650 713 809 875 935 94...,CC1(C(=O)NC(=O)N1C2=C(C=C(C=C2)C(F)(F)F)Cl)C,114 228 314 366 379 561 650 713 809 875 935 94...
9,c1ccc2c(c1)c(OCCN1CCCCC1)c1c3sc(nc32)S1,2 13 80 131 162 243 286 326 333 378 533 561 67...,C1CCN(CC1)CCOC2=C3C(=NC4=C2SC5=C4C=CC=C5C6=C(C...,2 13 31 80 131 137 162 277 333 352 378 461 533...


## 3. Càlcul del Tanimoto coefficient entre cada parell de SMILES per tots els fingerprints

In [1]:
for col in get_supported_fingerprints():
    
    fingerprints_input = []
    # Apliquem el conversor SMILES -> fingerprint
    fingerprints_input = df[SMILES_in_col_name].apply(
        lambda s: smiles_to_fingerprint(s, fp_type=col, n_bits=2048, return_bits=False)
    )

    fingerprints_output = []
    # Apliquem el conversor SMILES -> fingerprint
    fingerprints_output = df[SMILES_out_col_name].apply(
        lambda s: smiles_to_fingerprint(s, fp_type=col, n_bits=2048, return_bits=False)
    )

    # Considerem un SMILES Invàlid com que té Tanimoto 0
    tanimotos_Inv_0 = [
        DataStructs.TanimotoSimilarity(fp_in, fp_out)*100 # multipliquem per 100 per tenir-ho en percentatge
        # guardem 0 en els casos on l'output no és correcte i NaN si l'input és NaN
        if (not pd.isna(fp_out) and fp_out != "InvalidSMILE")
        else (0 if fp_out != "InvalidSMILE" else np.nan)
        for fp_in, fp_out in zip(fingerprints_input, fingerprints_output)
    ]

    # No Considerem els SMILES Invàlids
    tanimotos_Inv_NaN = [
        DataStructs.TanimotoSimilarity(fp_in, fp_out)*100 # multipliquem per 100 per tenir-ho en percentatge
        # guardem NaN en els casos on l'output no és correcte i NaN si l'input és NaN
        if (not pd.isna(fp_out) and fp_out != "InvalidSMILE")
        else (np.nan if fp_out != "InvalidSMILE" else np.nan)
        for fp_in, fp_out in zip(fingerprints_input, fingerprints_output)
    ]

    df[col+"Inv_0"] = tanimotos_Inv_0
    df[col+"Inv_NaN"] = tanimotos_Inv_NaN

NameError: name 'get_supported_fingerprints' is not defined

In [11]:
avg_tanimotos_Inv_0 = {col: df[col+"Inv_0"].mean() for col in get_supported_fingerprints()}
Tanimoto_Inv_0_df = pd.DataFrame([avg_tanimotos_Inv_0], index=[fp_type])

Tanimoto_Inv_0_df

,MACCS,Avalon,RDK4,RDK4-L,HashAP,TT,HashTT,ECFP0,ECFP2,ECFP4,FCFP2,FCFP4,AEs,ECFP2*,ECFP4*
ECFP4,92.659027,83.864632,82.799149,83.3164,77.702219,77.556783,77.635065,86.049497,82.657293,78.19532,82.872258,79.069226,82.563077,90.269735,81.919129


In [ ]:
avg_tanimotos_Inv_NaN = {col: df[col+"Inv_0"].mean() for col in get_supported_fingerprints()}
Tanimoto_Inv_NaN_df = pd.DataFrame([avg_tanimotos_Inv_NaN], index=[fp_type])

Tanimoto_Inv_NaN_df

## 4. Càlcul d'altres paràmetres interessants

### Average tanimoto score

In [12]:
Tanimoto_Inv_0_df["Avg_Tc"] = Tanimoto_df.mean(axis=1)
Tanimoto_Inv_NaN_df["Avg_Tc"] = Tanimoto_df.mean(axis=1)

### Percentatge d'al·lucinacions de MolForge

In [13]:
Tanimoto_Inv_0_df["Invalid (%)"] = (df[fp_out_col_name] == "InvalidSMILE").mean() * 100
Tanimoto_Inv_NaN_df["Invalid (%)"] = (df[fp_out_col_name] == "InvalidSMILE").mean() * 100

### Percentatge de string-exacts

In [14]:
Tanimoto_Inv_0_df["String-exacts (%)"] = (df["ECFP4"] == 100).mean() * 100
Tanimoto_Inv_NaN_df["String-exacts (%)"] = (df["ECFP4"] == 100).mean() * 100

## 5. Guardar el dataframe

In [ ]:
Tanimoto_Inv_0_df
Tanimoto_Inv_0_df.to_csv(output_path+"Tanimoto_Inv_0.csv")

In [15]:
Tanimoto_Inv_NaN_df
Tanimoto_Inv_NaN_df.to_csv(output_path+"Tanimoto_Inv_NaN.csv")

,MACCS,Avalon,RDK4,RDK4-L,HashAP,TT,HashTT,ECFP0,ECFP2,ECFP4,FCFP2,FCFP4,AEs,ECFP2*,ECFP4*,Avg_Tc,Invalid (%),String-exacts (%)
ECFP4,92.659027,83.864632,82.799149,83.3164,77.702219,77.556783,77.635065,86.049497,82.657293,78.19532,82.872258,79.069226,82.563077,90.269735,81.919129,82.608587,4.9,44.7
